Below is an example of the sift algorithm, with the trilinear interpolation smoothing of a histogram tensor of the generate descriptors being implemented in hardware using our overlay, histogram_tensor

This overlay streams in inputs for the histogram tensor calculations, which are done on the board, and the resulting structure is stream back out and placed back into the software structure for further calculations

In [1]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/pynq/overlays/descriptor/design_1.bit')
dma = overlay.axi_dma_0

### Code to find keypoints and descriptors for the first image

In [ ]:
from sfm_algo_unpacked import generateBaseImage, computeNumberOfOctaves, generateGaussianKernels, generateGaussianImages, generateDoGImages, findScaleSpaceExtrema, removeDuplicateKeypoints, convertKeypointsToInputImageSize, generateDescriptors, generateDescriptors_hardware
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2

img1 = cv2.imread('dino_data/dino02.jpg', cv2.IMREAD_GRAYSCALE)

sigma=1.6 
num_intervals=3 
assumed_blur=0.5 
image_border_width=5

base_image = generateBaseImage(img1, sigma, assumed_blur)
print("Finished base_image")
num_octaves = computeNumberOfOctaves(base_image.shape)
print("Finished num_octaves")
gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
print("Finished gaussian_kernels")
gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
print("Finished gaussian_images")
dog_images = generateDoGImages(gaussian_images)
print("Finished dog_images")
keypoints = findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width)
print("Finished keypoints")
keypoints = removeDuplicateKeypoints(keypoints)
print("Finished remove duplicate keypoints")
kp1 = convertKeypointsToInputImageSize(keypoints)
print("Finished convert to image size")
des1 = generateDescriptors_hardware(keypoints, gaussian_images, dma)
print("Finished generate_descriptors")

### Code to find keypoints and descriptors for the second image

In [ ]:
from sfm_algo_unpacked import generateBaseImage, computeNumberOfOctaves, generateGaussianKernels, generateGaussianImages, generateDoGImages, findScaleSpaceExtrema, removeDuplicateKeypoints, convertKeypointsToInputImageSize, generateDescriptors, generateDescriptors_hardware
import numpy as np
import matplotlib.pyplot as plt
import time
import cv2

img2 = cv2.imread('dino_data/dino05.jpg', cv2.IMREAD_GRAYSCALE)

sigma=1.6 
num_intervals=3 
assumed_blur=0.5 
image_border_width=5

base_image = generateBaseImage(img1, sigma, assumed_blur)
print("Finished base_image")
num_octaves = computeNumberOfOctaves(base_image.shape)
print("Finished num_octaves")
gaussian_kernels = generateGaussianKernels(sigma, num_intervals)
print("Finished gaussian_kernels")
gaussian_images = generateGaussianImages(base_image, num_octaves, gaussian_kernels)
print("Finished gaussian_images")
dog_images = generateDoGImages(gaussian_images)
print("Finished dog_images")
keypoints = findScaleSpaceExtrema(gaussian_images, dog_images, num_intervals, sigma, image_border_width)
print("Finished keypoints")
keypoints = removeDuplicateKeypoints(keypoints)
print("Finished remove duplicate keypoints")
kp2 = convertKeypointsToInputImageSize(keypoints)
print("Finished convert to image size")
des2 = generateDescriptors_hardware(keypoints, gaussian_images, dma)
print("Finished generate_descriptors")

### Code to run to show the found keypoints and match them

In [ ]:
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Lowe's ratio test
good = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good.append(m)


# Estimate homography between template and scene
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

M = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)[0]

# Draw detected template in scene image
h, w = img1.shape
pts = np.float32([[0, 0],
                    [0, h - 1],
                    [w - 1, h - 1],
                    [w - 1, 0]]).reshape(-1, 1, 2)
dst = cv2.perspectiveTransform(pts, M)

img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

h1, w1 = img1.shape
h2, w2 = img2.shape
nWidth = w1 + w2
nHeight = max(h1, h2)
hdif = int((h2 - h1) / 2)
newimg = np.zeros((nHeight, nWidth, 3), np.uint8)

for i in range(3):
    newimg[hdif:hdif + h1, :w1, i] = img1
    newimg[:h2, w1:w1 + w2, i] = img2

# Draw SIFT keypoint matches
for m in good:
    pt1 = (int(kp1[m.queryIdx].pt[0]), int(kp1[m.queryIdx].pt[1] + hdif))
    pt2 = (int(kp2[m.trainIdx].pt[0] + w1), int(kp2[m.trainIdx].pt[1]))
    cv2.line(newimg, pt1, pt2, (255, 0, 0))

plt.imshow(newimg)
plt.show()

### Code to turn keypoints and descriptors into point cloud

In [ ]:
from point_cloud import compute_essential_normalized, compute_P_from_essential, reconstruct_one_point, linear_triangulation
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

good = []
for m, n in matches:
    if m.distance < 0.8 * n.distance:
        good.append(m)

src_pts = np.asarray([kp1[m.queryIdx].pt for m in good])
dst_pts = np.asarray([kp2[m.trainIdx].pt for m in good])

retval, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 100.0)
mask = mask.ravel()

pts1 = src_pts[mask == 1]
pts2 = dst_pts[mask == 1]

height, width = img1.shape
intrinsic = np.array([  # for dino
    [2360, 0, width / 2],
    [0, 2360, height / 2],
    [0, 0, 1]])

points1n = np.dot(np.linalg.inv(intrinsic), pts1.T)
points2n = np.dot(np.linalg.inv(intrinsic), pts2.T)

E = compute_essential_normalized(points1n, points2n)
print('Computed essential matrix:', (-E / E[0][1]))


P1 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
P2s = compute_P_from_essential(E)

ind = -1
for i, P2 in enumerate(P2s):
    d1 = reconstruct_one_point(
        points1n[:, 0], points2n[:, 0], P1, P2)

    P2_homogenous = np.linalg.inv(np.vstack([P2, [0, 0, 0, 1]]))
    d2 = np.dot(P2_homogenous[:3, :4], d1)

    if d1[2] > 0 and d2[2] > 0:
        ind = i


start = time.time()
P2 = np.linalg.inv(np.vstack([P2s[ind], [0, 0, 0, 1]]))[:3, :4]
tripoints3d = linear_triangulation(points1n, points2n, P1, P2)




fig = plt.figure()
fig.suptitle('3D reconstructed', fontsize=16)
ax = fig.add_subplot(111, projection='3d')
ax.plot(tripoints3d[0], tripoints3d[1], tripoints3d[2], 'b.')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
ax.view_init(elev=135, azim=90)
plt.show()




